In [16]:
import torch
from utils.transforms import reshape_transform, flattening_transform, create_flattening_index_set
from utils.supervised_dataset import supervised_dataset as dataset

train_set_input_dir = "data/example_3_1/training_set/inputs/"
train_set_target_dir = "data/example_3_1/training_set/target_values/"
test_set_input_dir = "data/example_3_1/test_set/inputs/"
test_set_target_dir = "data/example_3_1/test_set/target_values/"
device = 'mps'
H,W = 32,32


target_transform = reshape_transform(H,W)
transform = None

train_set = dataset(
    input_dir=train_set_input_dir, 
    target_dir=train_set_target_dir, 
    transform=transform, 
    target_transform=target_transform, 
    device=device
)

test_set = dataset(
    input_dir=test_set_input_dir, 
    target_dir=test_set_target_dir, 
    transform=transform, 
    target_transform=target_transform, 
    device=device
)

x,t = test_set[1]
fn = torch.nn.Conv2d(in_channels=13,out_channels=2,kernel_size=(5,5), padding=2,dtype=torch.float32, device='mps')
fn(x).shape
x.shape


torch.Size([13, 34, 34])

In [47]:
import torch
from utils.transforms import reshape_transform, flattening_transform, create_flattening_index_set
from utils.supervised_dataset import supervised_dataset as dataset

train_set_input_dir = "data/example_3_1/training_set/inputs/"
train_set_target_dir = "data/example_3_1/training_set/target_values/"
test_set_input_dir = "data/example_3_1/test_set/inputs/"
test_set_target_dir = "data/example_3_1/test_set/target_values/"
device = 'mps'
H,W = 32,32


target_transform = flattening_transform(torch.arange(H*W), 1, True)
transform = flattening_transform(create_flattening_index_set(H,W,False),13, True)

train_set = dataset(
    input_dir=train_set_input_dir, 
    target_dir=train_set_target_dir, 
    transform=transform, 
    target_transform=target_transform, 
    device=device
)

test_set = dataset(
    input_dir=test_set_input_dir, 
    target_dir=test_set_target_dir, 
    transform=transform, 
    target_transform=target_transform, 
    device=device
)

x1,t1 = test_set[1]
fn1 = torch.nn.Conv1d(in_channels=1,out_channels=1,kernel_size=1013,dtype=torch.float32, device='mps')
fn2 = torch.nn.AdaptiveMaxPool1d(3000)
print(t1.shape, fn2(x1).shape)

torch.Size([1, 1024]) torch.Size([1, 3000])


In [41]:
8*1024

8192

In [17]:
import torch
from utils.transforms import reshape_transform, flattening_transform, create_flattening_index_set, channeled_flattening_transform
from utils.supervised_dataset import supervised_dataset as dataset

train_set_input_dir = "data/example_3_1/training_set/inputs/"
train_set_target_dir = "data/example_3_1/training_set/target_values/"
test_set_input_dir = "data/example_3_1/test_set/inputs/"
test_set_target_dir = "data/example_3_1/test_set/target_values/"
device = 'mps'
H,W = 32,32


target_transform = flattening_transform(torch.arange(H*W), 1, True)
transform = channeled_flattening_transform(create_flattening_index_set(H+2,W+2,False),13)

train_set = dataset(
    input_dir=train_set_input_dir, 
    target_dir=train_set_target_dir, 
    transform=transform, 
    target_transform=target_transform, 
    device=device
)

test_set = dataset(
    input_dir=test_set_input_dir, 
    target_dir=test_set_target_dir, 
    transform=transform, 
    target_transform=target_transform, 
    device=device
)

x1,t1 = test_set[1]
fn1 = torch.nn.Conv1d(in_channels=13,out_channels=2,kernel_size=25, padding=12,dtype=torch.float32, device='mps')
print(fn1(x1).shape, x1.shape)

torch.Size([2, 1156]) torch.Size([13, 1156])


In [31]:
from utils.transforms import create_cell_ind_to_grid

ctg = create_cell_ind_to_grid(10,5)
ctg


tensor([[35, 40, 44, 47, 49],
        [30, 36, 41, 45, 48],
        [25, 31, 37, 42, 46],
        [20, 26, 32, 38, 43],
        [15, 21, 27, 33, 39],
        [10, 16, 22, 28, 34],
        [ 6, 11, 17, 23, 29],
        [ 3,  7, 12, 18, 24],
        [ 1,  4,  8, 13, 19],
        [ 0,  2,  5,  9, 14]])

In [ ]:
from utils.transforms import cells_from_flat_array_ind
class cnn1d(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.net = torch.nn.Sequential(
            torch.nn.Conv1d(in_channels=13,out_channels=9,kernel_size=9, padding=4,dtype=torch.float32, device='mps'),
            torch.nn.LeakyReLU(),
            torch.nn.Conv1d(in_channels=9,out_channels=3,kernel_size=9, padding=4,dtype=torch.float32, device='mps'),
            torch.nn.LeakyReLU(),
            torch.nn.Conv1d(in_channels=3,out_channels=1,kernel_size=9, padding=4,dtype=torch.float32, device='mps')
        )
        self.idx_key = cells_from_flat_array_ind(34,34)

    def forward(self, x):
        z = self.net(x)
        return z[:,self.idx_key+4]

model1d=cnn1d()
print(torch.nn.MSELoss()(model1d(x1), t1))



tensor(0.8008, device='mps:0', grad_fn=<MseLossBackward0>)


tensor([ 4,  7,  8, 11])

In [15]:
ctg[:,0].flip(0)

tensor([ 0,  1,  3,  6, 10, 15, 21, 28])